# Toy problem: retrieve infeasibility info from gurobi when solving QPs

Consider the infeasible QP
\begin{align}
\min \frac{1}{2} x_1^2
\quad
\text{subject to}
\quad
x_0 \geq 0,
x_1 \geq 0,
x_0 + x_1 \leq -1,
\end{align}
and its unbounded dual
\begin{align}
\max - \frac{1}{2} (p_1-q)^2 + q
\quad
\text{subject to}
\quad
p_0 \geq 0,
p_1 \geq 0,
q \geq 0,
p_0 = q.
\end{align}

How to retrieve from Gurobi the multipliers $(p_0, p_1, q) = (\alpha, \alpha, \alpha)$ with $\alpha > 0$ which prove unboundedness of the dual, and hence infeasibility of the primal?

It seems that this info can be retrieved only in case of linear programs.
However, the same multipliers that prove infeasibility of the LP also prove infeasibility of the QP.
Hence it is sufficient to set to objective to zero and run the LP.

Note that the dual of the LP
\begin{align}
\min 0
\quad
\text{subject to}
\quad
x_0 \geq 0,
x_1 \geq 0,
x_0 + x_1 \leq -1,
\end{align}
is
\begin{align}
\max q
\quad
\text{subject to}
\quad
p_0 \geq 0,
p_1 \geq 0,
q \geq 0,
p_0 = q,
p_1 = q.
\end{align}

In [130]:
import gurobipy as grb

In [140]:
m = grb.Model()
x = m.addVars(2, lb=[-grb.GRB.INFINITY]*2)
m.setObjective(x[0] + x[1])

# bounds are expressed as constraints otherwise we cannot get their multipliers out of the Farkas proof
lb0 = m.addConstr(- x[0] <= 0., name='') # express the constraints with <= so that multipliers are positive
lb1 = m.addConstr(- x[1] <= 0.)
c = m.addConstr(x[0] + x[1] <= 1.)
c_redundant = m.addConstr(- x[0] - x[1] <= 1.) # redundant

In [141]:
m.Params.InfUnbdInfo = 1 # to query the infeasibility proof (only LP)
m.optimize()

Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 2 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  0.000000000e+00


In [142]:
# retrieve duals which prove infeasibility for the LP
print m.FarkasDual

GurobiError: Unable to retrieve attribute 'FarkasDual'

In [143]:
# note that calling the duals directly gives garbage
print [lb0.Pi, lb1.Pi, c.Pi, c_redundant.Pi]

[-1.0, -1.0, 0.0, 0.0]


In [121]:
m.getConstrByName('')

In [122]:
from collections import OrderedDict

In [123]:
constraints = OrderedDict()

In [124]:
constraints['gatto'] = 1

In [125]:
constraints

OrderedDict([('gatto', 1)])

In [126]:
aaa = ('u', 22, 3)

In [96]:
lb0.RHS = 22.
m.update()

In [98]:
lb0.

<gurobi.Constr R0>

In [102]:
x = m.addVars(10, name='x')
m.update()

In [110]:
lb0.Pi

AttributeError: Unable to retrieve attribute 'Pi'